# Recursive Clustering and Summarization

Plan:
- recursively cluster collections
- create tree of clusters (the HDBSCAN does this anyways but likely not as we want)
- cluster until max-depth is reached or (better) until each leaf only has one "plausible" cluster (based on thresholds or probabilities)
- try summarizing to get "main idea" out of cluster


- cluster on keywords ( randomize all grammar + stop words )
- topic clusters
- context: title, abstract, etc. keywords


## TODO 28 01 2022

* Run topic clustering (BERT) over cluster_tree
* hook back sentences to DOI 


## Recursively cluster 

Based on the topic_clustering notebook, we will try with Agglomerative Clustering 

In [1]:
import pandas as pd


df = pd.read_csv("downloads/25k_enriched_small.csv")

In [2]:
df2 = pd.read_csv("downloads/core_pos_min.csv")

In [3]:
df2["text"] = df2["first_sent"]

In [4]:
df2["source"] = "core_pubmed"

In [5]:
df3 = df.append(df2)

In [6]:
df = df3

In [7]:
df["oai"]

0                                      NaN
1                                      NaN
2                                      NaN
3                                      NaN
4                                      NaN
                       ...                
42196    oai:pubmedcentral.nih.gov:2662567
42197    oai:pubmedcentral.nih.gov:3291942
42198    oai:pubmedcentral.nih.gov:2466601
42199    oai:pubmedcentral.nih.gov:3302717
42200    oai:pubmedcentral.nih.gov:3356029
Name: oai, Length: 66547, dtype: object

In [8]:
df = df.sample(frac=1)

In [9]:
sentences = list(df["text"])

In [10]:
df.to_csv("downloads/60k_core_all.csv",  index = False, header=True)

In [3]:
!pip install metapub

  Using cached metapub-0.5.5.tar.gz (120 kB)
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 6.4 MB 27.7 MB/s            
  Using cached eutils-0.6.0-py2.py3-none-any.whl (41 kB)
  Using cached habanero-1.0.0-py2.py3-none-any.whl (42 kB)
  Using cached cssselect-1.1.0-py2.py3-none-any.whl (16 kB)
  Using cached Unidecode-1.3.2-py3-none-any.whl (235 kB)
  Using cached tox-3.24.5-py2.py3-none-any.whl (85 kB)
     |████████████████████████████████| 280 kB 126.9 MB/s            
  Using cached coloredlogs-15.0.1-py2.py3-none-any.whl (46 kB)
  Using cached python-Levenshtein-0.12.2.tar.gz (50 kB)
  Preparing metadata (setup.py) ... done
  Using cached humanfriendly-10.0-py2.py3-none-any.whl (86 kB)
     |████████████████████████████████| 98 kB 16.0 MB/s             
  Using cached virtualenv-20.13.0-py2.py3-none-any.whl (6.5 MB)
  Using cached distlib-0.3.4-py2.py3-none-any.whl (461 kB)
  Created wheel for metapub: filename=metapub-0.5.5-py3-none-any.whl siz

In [8]:
#pos = df[df.label == 1]
sentences = list(df["text"]) #otherwise key error

In [10]:
df[df["PMID"].notna()][:100]["PMID"]

0     23301433.0
1     26136407.0
2     32005202.0
3     33044104.0
4     33027267.0
         ...    
95    26475960.0
96    29801496.0
97    17030353.0
98    30693884.0
99    33972143.0
Name: PMID, Length: 100, dtype: float64

In [61]:
from metapub.convert import pmid2doi
pmid2doi(17030353)
def safe_pmid2doi(pmid):
    try:
        pmid2doi(pmid)
    except:
        print(pmid, " error")
        return None

2022-02-01 02:29:30 ip-172-16-67-41 metapub.config[14143] WARNING NCBI_API_KEY was not set.


In [23]:
!pip install sentence-transformers

  Using cached sentence-transformers-2.2.0.tar.gz (79 kB)
  Preparing metadata (setup.py) ... done
  Using cached transformers-4.17.0-py3-none-any.whl (3.8 MB)
     |████████████▊                   | 351.3 MB 144.0 MB/s eta 0:00:04

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



     |██████████████████████████▋     | 733.8 MB 869 kB/s eta 0:02:51  

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



     |████████████████████████████████| 881.9 MB 6.2 kB/s              
     |████████████████████████████████| 23.3 MB 24.6 MB/s            
     |████████████████████████████████| 1.2 MB 85.0 MB/s            
  Using cached huggingface_hub-0.4.0-py3-none-any.whl (67 kB)
     |████████████████████████████████| 6.5 MB 127.9 MB/s            
  Using cached sacremoses-0.0.47-py2.py3-none-any.whl (895 kB)
     |████████████▌                   | 344.6 MB 158.5 MB/s eta 0:00:04

...

     |█████████████████████████▋      | 706.8 MB 155.7 MB/s eta 0:00:02

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



     |██████████████████████████████▏ | 830.8 MB 161.7 MB/s eta 0:00:01

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



     |████████████████████████████████| 881.9 MB 7.3 kB/s              
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=04cc938b9545d3dd87dc08bc4c892ced358c5fc6369c7c948c15e0349a21af3e
  Stored in directory: /home/ec2-user/.cache/pip/wheels/ce/2d/86/18ade2fbe66e17694cf8f5667a68d2bd06a65bce42546ea675
Successfully built sentence-transformers


In [32]:
from sentence_transformers import SentenceTransformer, util

print("Encode the corpus ... get a coffee in the meantime")
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
embeddings = model.encode(sentences, batch_size=64, show_progress_bar=True, convert_to_tensor=True)

Encode the corpus ... get a coffee in the meantime


Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.1k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/363 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/1040 [00:00<?, ?it/s]

In [35]:
from sklearn.cluster import AgglomerativeClustering
import numpy as np

def cluster(embeddings, **kwargs):
    embeddings = embeddings.cpu()
    # Normalize the embeddings to unit length
    corpus_embeddings = embeddings /  np.linalg.norm(embeddings, axis=1, keepdims=True)

    # Perform kmean clustering
    clustering_model = AgglomerativeClustering(**kwargs) #, affinity='cosine', linkage='average', distance_threshold=0.4)
    clustering_model.fit(corpus_embeddings)
   # cluster_assignment = clustering_model.labels_
    return clustering_model

In [36]:
def get_clusters(clustering_model): 
    clusters = {}
    for sentence_id, cluster_id in enumerate(clustering_model.labels_):
        if cluster_id not in clusters:
            clusters[cluster_id] = []
        try:
            clusters[cluster_id].append(sentences[sentence_id])
        except:
            print(sentence_id, "sentence_id")
    return clusters
   
#     for i, cluster in clustered_sentences.items():
#         print("Cluster ", i+1)
#         print(cluster)
#         print("\n")

In [37]:
sample = embeddings

In [38]:
cluster_model = cluster(sample, n_clusters=None, distance_threshold=1.4)

Cluster Model attributes

   n_clusters_ : int
        The number of clusters found by the algorithm. If
        ``distance_threshold=None``, it will be equal to the given
        ``n_clusters``.

    labels_
    n_leaves_

    n_connected_components_ : The estimated number of connected components in the graph.

    children_ : array-like of shape (n_samples-1, 2)
        The children of each non-leaf node. Values less than `n_samples`
        correspond to leaves of the tree which are the original samples.
        A node `i` greater than or equal to `n_samples` is a non-leaf
        node and has children `children_[i - n_samples]`. Alternatively
        at the i-th iteration, children[i][0] and children[i][1]
        are merged to form node `n_samples + i`

    distances_ : array-like of shape (n_nodes-1,)
        Distances between nodes in the corresponding place in `children_`.
        Only computed if `distance_threshold` is used or `compute_distances`
        is set to `True`.

In [39]:
len(get_clusters(cluster_model).keys())

6362

In [44]:
clusters = get_clusters(cluster_model)

In [48]:
def collection_to_clusters(texts, model=model,  **kwargs):
    embs = model.encode(texts, batch_size=64, show_progress_bar=True, convert_to_tensor=True)
    cluster_model = cluster(embs, **kwargs)
    return get_clusters(cluster_model).values()

In [49]:
from tqdm import tqdm

In [50]:
cluster_tree = {}
lens = [len(v) for v in clusters.values()]
for i, v in tqdm(clusters.items()):
    if len(v) > 25:
        cluster_tree[i] = {"parent" : v}
        cluster_tree[i] = {"children" :[*collection_to_clusters(v, n_clusters=None, distance_threshold=0.7)]}
        #get values, embed and sample again with lower threshold      

  0%|          | 0/6362 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 3/6362 [00:00<06:25, 16.49it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 5/6362 [00:00<06:59, 15.15it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

 82%|████████▏ | 5193/6362 [00:37<00:02, 435.73it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 6362/6362 [00:37<00:00, 168.33it/s]


In [43]:
len(get_clusters(cluster_model).keys())

6362

In [51]:
import json

ct = {str(k):v for k,v in cluster_tree.items()}
with open('cluster_tree_65k_tresh_1,4.json', 'w') as outfile:
    json.dump(ct, outfile)

In [36]:
!pip bertopic --version

ERROR: unknown command "bertopic"


In [239]:
df.to_csv("25k_pm_enriched.csv", index = False, header=True)

In [236]:
df["DOI"] = df["PMID"].apply(pmid)

,level_0,sentence_id,title,article_link,text,year,mesh_list,journal_title,volume,issue,...,sentences,first_sent,predicts,text_masked,text_length,sent_id,cluster_id,topic_id,probs,topic_name
5,5,5,NaN,NaN,"Over the past decade, there are significant ad...",NaN,NaN,NaN,NaN,NaN,...,"['Over the past decade, there are significant ...","Over the past decade, there are significant ad...",1.0,NaN,NaN,5,2416,-1,"[12, 292, 418]",cervical cancer women
15,15,15,NaN,NaN,Cervical cancer is the fourth most common canc...,NaN,NaN,NaN,NaN,NaN,...,['Cervical cancer is the fourth most common ca...,Cervical cancer is the fourth most common canc...,1.0,NaN,NaN,15,1462,12,"[12, 418, 292]",cervical cancer women
16,16,16,NaN,NaN,Cervical cancer is one of the most common form...,NaN,NaN,NaN,NaN,NaN,...,['Cervical cancer is one of the most common fo...,Cervical cancer is one of the most common form...,1.0,NaN,NaN,16,1462,12,"[12, 292, 418]",cervical cancer women
34,34,34,NaN,NaN,Cervical cancer is one of the commonest neopla...,NaN,NaN,NaN,NaN,NaN,...,['Cervical cancer is one of the commonest neop...,Cervical cancer is one of the commonest neopla...,1.0,NaN,NaN,34,1937,12,"[12, 292, 418]",cervical cancer women
45,45,45,NaN,NaN,Cervical cancer is an important health problem...,NaN,NaN,NaN,NaN,NaN,...,['Cervical cancer is an important health probl...,Cervical cancer is an important health problem...,1.0,NaN,NaN,45,2416,12,"[12, 292, 418]",cervical cancer women
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24341,22293,24341,NaN,NaN,These findings suggest that fall-related injur...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,These findings suggest that fall iihyph relate...,21.0,24341,367,27,"[27, 3, 85]",falls fall older
24342,22294,24342,NaN,NaN,Bacteriuria -- asymptomatic and symptomatic --...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,iinnp ii: iijj and symptomatic ii: always has ...,23.0,24342,2460,298,"[298, 300, 403]",catheter catheters pn
24343,22295,24343,NaN,NaN,Strategies and solutions to a vexing problem .,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Strategies and solutions to a iijj problem .,8.0,24343,1036,-1,"[250, 434, 86]",mcmaster believe educating
24344,22296,24344,NaN,NaN,Peripheral nerve injury ( PNI ) is a serious p...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Peripheral nerve injury -lrb- iinnp -rrb- is a...,23.0,24344,974,344,"[344, 164, 301]",nerve peripheral peroneal


In [50]:
td = extract_topics(sents)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2022-01-31 00:54:25,580 - BERTopic - Reduced dimensionality with UMAP
2022-01-31 00:54:25,582 - BERTopic - Predicted clusters with HDBSCAN
2022-01-31 00:54:25,585 - BERTopic - Calculated probabilities with HDBSCAN


In [ ]:
import json

with open("full_topic_triples.json", "w") as f:
    json.dump(td, f)

## Get two top probability words
* get third one as test
* link up sentences to their topic AND the cluster
* df = sentenceID, DOI, clusterID, clusterTOPICs

In [52]:
def get_cluster_for_sent(sent, clusterdict):
    for idx, sents in clusterdict.items():
        if sent in sents:
            return idx
    return -1

In [ ]:
n_topics = 470
cluster_model_2 = cluster(sample, n_clusters=n_topics)# distance_threshold=1.4)
clusters_2 = get_clusters(cluster_model)

In [54]:
#get cluster of sentence
df["cluster_id"] = df["text"].apply(lambda t: get_cluster_for_sent(t, clusters))
#get topic of sentence

## FAISS lookup


In [241]:
emb = embeddings.cpu()

In [249]:
emb = emb.numpy()

In [250]:
d = emb.shape[1]

In [300]:
emb.shape, len(df)

((24346, 768), 24346)

In [343]:
np.array(df["emb"][1])[:].shape

(768,)

In [304]:
x = [list(e) for e in emb ]

In [305]:
df["emb"] = x

,level_0,problem_id,title,article_link,text,year,mesh_list,journal_title,volume,issue,...,first_sent,predicts,text_masked,text_length,sent_id,cluster_id,topic_id,probs,topic_name,emb
0,0,0,NaN,NaN,The high cervical cancer mortality rate among ...,NaN,NaN,NaN,NaN,NaN,...,The high cervical cancer mortality rate among ...,1.0,NaN,NaN,0,2191,12,"[12, 292, 418]",cervical cancer women,"[0.0044709765, 0.09916535, -0.024515742, -0.07..."
1,1,1,NaN,NaN,The clinical management of cervical cancer in ...,NaN,NaN,NaN,NaN,NaN,...,The clinical management of cervical cancer in ...,1.0,NaN,NaN,1,334,12,"[12, 292, 418]",cervical cancer women,"[0.05009258, 0.036560223, -0.023339115, -0.043..."
2,2,2,NaN,NaN,Cervical cancer is almost entirely preventable...,NaN,NaN,NaN,NaN,NaN,...,Cervical cancer is almost entirely preventable...,1.0,NaN,NaN,2,2416,12,"[12, 292, 418]",cervical cancer women,"[0.03471848, 0.039798826, -0.00884888, -0.0790..."
3,3,3,NaN,NaN,Cervical cancer affects half a million women w...,NaN,NaN,NaN,NaN,NaN,...,Cervical cancer affects half a million women w...,1.0,NaN,NaN,3,2416,12,"[12, 292, 418]",cervical cancer women,"[0.03953646, 0.07881644, -0.035314273, -0.0663..."
4,4,4,NaN,NaN,Cervical cancer is one of the most common caus...,NaN,NaN,NaN,NaN,NaN,...,Cervical cancer is one of the most common caus...,1.0,NaN,NaN,4,2333,12,"[12, 292, 418]",cervical cancer women,"[0.068300106, 0.04014466, -0.040204357, -0.081..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24341,22293,24341,NaN,NaN,These findings suggest that fall-related injur...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,These findings suggest that fall iihyph relate...,21.0,24341,367,27,"[27, 3, 85]",falls fall older,"[-0.0552268, 0.015042118, 0.013946558, 0.01518..."
24342,22294,24342,NaN,NaN,Bacteriuria -- asymptomatic and symptomatic --...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,iinnp ii: iijj and symptomatic ii: always has ...,23.0,24342,2460,298,"[298, 300, 403]",catheter catheters pn,"[0.017118197, 0.056684177, -0.006661493, -0.04..."
24343,22295,24343,NaN,NaN,Strategies and solutions to a vexing problem .,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Strategies and solutions to a iijj problem .,8.0,24343,1036,-1,"[250, 434, 86]",mcmaster believe educating,"[0.07660182, 0.07224933, -0.008998395, -0.0116..."
24344,22296,24344,NaN,NaN,Peripheral nerve injury ( PNI ) is a serious p...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Peripheral nerve injury -lrb- iinnp -rrb- is a...,23.0,24344,974,344,"[344, 164, 301]",nerve peripheral peroneal,"[-0.056301344, 0.03706796, 0.01791248, 0.02877..."


In [60]:
def stepwise_l(step, df, f):
    l = []
    for idx in range(step, len(df)+step-1, step):
        print(idx-step, idx)# dataframe[idx-step:idx])
        sl = df[idx-step:idx]
        batch = [f(item) for item in sl]
        l = l + batch
    return l

In [376]:
x = stepwise_l(1000, df["emb"], find_closest_problem)

0 1000
1000 2000
2000 3000
3000 4000
4000 5000
5000 6000
6000 7000
7000 8000
8000 9000
9000 10000
10000 11000
11000 12000
12000 13000
13000 14000
14000 15000
15000 16000
16000 17000
17000 18000
18000 19000
19000 20000
20000 21000
21000 22000
22000 23000
23000 24000
24000 25000


In [59]:
import pandas as pd
df = pd.read_csv("downloads/25k_problems_enriched_2.csv", low_memory=False)

In [383]:
#df["closest_problem_id"] = x
#df["text"][[1,2,3]]

In [20]:
#[print(t) for t in df.text]

In [21]:
samp = df[:100]


In [24]:
df.text[[39, 53, 15682]]

39       U.S. Hispanic women suffer a disproportionate ...
53       Mexico has a high rate of cervical cancer whic...
15682    Uterine cervical cancer in Latin American wome...
Name: text, dtype: object

In [36]:
import re
test_string = '[39, 53, 15682]'
[int(item) for item in list(re.findall(r'\d+', test_string))]

[39, 53, 15682]

In [26]:
df["closest_problem_id"] = df["closest_problem_id"].astype(int)

'[39, 53, 15682]'

In [41]:
df["closest_problem_id"] = df["closest_problem_id"].apply(lambda id_str: [int(item) for item in list(re.findall(r'\d+', id_str))])

In [50]:
df = df.drop(columns=["text_masked",  ])

In [51]:
df = df.drop(columns=["Abstract",  ])

In [53]:
df = df.drop(columns=["sentences",  ])

In [57]:
df = df.drop(columns=["emb"])

In [58]:
df.to_csv("downloads/enrich_problem_id_25k_small.csv", index = False, header=True)

In [44]:
df["closest_problems"] = df["closest_problem_id"].apply(lambda ids: df["text"][ids])

ValueError: Wrong number of items passed 19604, placement implies 1

In [386]:
#df.to_csv("downloads/25k_problems_enriched_2.csv", index = False, header=True)

## Find DOIs

In [ ]:
dois = stepwise_l(300, df["PMID"][:10000], safe_pmid2doi)

0 300
33387427  error
17600938  error
16614679  error
300 600
15712471  error
8345190  error
6764395  error
19522274  error
4800 5100
928321  error
5100 5400
15503931  error
16669195  error
16155338  error
6368658  error
15813450  error
27434222  error
5400 5700
1292768  error
2390366  error
18802217  error
10940864  error
5700 6000
26742229  error
11811271  error
6000 6300
6300 6600
20586161  error
17679382  error
24716360  error
17679377  error
33959168  error
1528367  error
6600 6900
20380253  error
32038013  error
6900 7200
11760460  error
19920288  error
21574131  error
5869753  error
16958232  error
22708311  error
1935683  error
2408189  error
7200 7500
5874186  error
24724435  error
5877223  error
19591099  error
8140755  error
7753535  error
19112838  error
11766326  error
7500 7800
27824311  error
23995203  error
33263294  error
16225089  error
16583327  error
12572263  error
11984950  error
9431131  error
10249184  error
7800 8100
19893863  error
21192484  error
17966899  er

In [76]:
[doi for doi in dois if doi]

[]

In [ ]:
df.to_csv("downloads/25k_problems_enriched_3.csv", index = False, header=True)

In [310]:
df["problem_id"].map(type)

0        <class 'int'>
1        <class 'int'>
2        <class 'int'>
3        <class 'int'>
4        <class 'int'>
             ...      
24341    <class 'int'>
24342    <class 'int'>
24343    <class 'int'>
24344    <class 'int'>
24345    <class 'int'>
Name: problem_id, Length: 24346, dtype: object

In [312]:
df[df["problem_id"] == 19652]

,level_0,problem_id,title,article_link,text,year,mesh_list,journal_title,volume,issue,...,first_sent,predicts,text_masked,text_length,sent_id,cluster_id,topic_id,probs,topic_name,emb
19652,17473,19652,NaN,NaN,To enhance clinical outcome of successful T ce...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,To enhance clinical outcome of successful T ce...,39.0,19652,334,-1,"[111, 374, 221]",antibodies vaccines antigens,"[0.03265389, -0.03944448, 0.0027229234, -0.017..."


(1, 768)

[[    1 19652    56  8344]]
[[0.         0.43545723 0.45587155 0.4650954 ]]


## Hierachy by source text overlap


In [ ]:
#take td
#if there's overlap of t1 and t2 t1 is parent if t2 has less of the sources (and 80%+) of t2 sources are in t1 (otherwise duplicate or not that realted))

In [ ]:
def sources_relation(s1, s2, min_overlap=0.7):
    s1 = set(s1)
    s2 = set(s2)
    l1 = len(s1)
    l2 = len(s2)

    common = s1.intersection(s2) #order doesn't matter
    
#     cl1 = len(s1.intersect(common))
#     cl2 = len(s2.intersect(common))
    
    relation="TBA"
    #if only a few sources are common and it's not all/most of one topics sources then -> different ideas
    #if many are common and one topic is much bigger, the smaller is a subtopic
    #if almost all are common and there's 50:50 or 30:70 split then they are duplicates
    #either unrelated, duplicate, or hypernomy
    
    cl = len(common)

    ol2 = cl/l2
    ol1 = cl/l1
    if l1 > l2:
        ol = ol2
    else:
        ol = ol1
    
    if ol < min_overlap: #little in common (one topic is enough to decide)
        relation="different"
        
        #-- comparative
    if ol > min_overlap: #if lots of overlap
        if l2/l1 < 8/10:
            relation="child"
            
            #TODO: use permutations and then filter out duplicates / differents ... (only child left)
        elif l1/l2 < 8/10:
            relation="parent"
        else: #approx similar size
            #topics similar size and much of overlap (synonyms)
            relation = "duplicates"
    
    return relation


In [ ]:
from itertools import combinations

topic_pairs = combinations(td.keys(), 2)
result = [(t1, t2, sources_relation(td[t1], td[t2]) ) for t1,t2 in topic_pairs]


### Building the Tree

In [ ]:
triples = [r for r in result if r[2] == "child" or r[2] == "parent"]

In [ ]:
triples[:8], len(result), len(triples)

In [ ]:
#build the tree

def tree_has_topic(topic, tree):
    return len([k for k in FlatterDict(tree).keys() if topic in k]) > 0

def get_leaves(topic, triples):
    node = {}
    node[topic] = {"children": []}
    #topic = node
    for t in triples:
        if (t[0] == topic and t[2] == "child") and not tree_has_topic(t[1], triples): #":or (t[1] == topic and t[2] == "parent")) 
            node[topic]["children"] += [get_leaves(t[1], triples)] 
            
            # TODO: flatten and check if key is already in 
    if len(node[topic]["children"]) == 0: # leave
        return node
    return node

In [ ]:
x = "reperfusion"
[t for t in triples if t[0] == x or t[1] == x]

In [ ]:
from flatdict import FlatDict, FlatterDict
y = get_leaves("ischemia", triples)

y

In [ ]:
triples[:20]

In [ ]:
trees = {}
for triple in triples:
    #every entity first get's it's children attached before attaching to parent
    topic = triple[0]
    if not trees.get(topic):
        trees[topic] = get_leaves(topic, triples)[topic]

trees

In [ ]:
for vs in trees["reperfusion"].values():
    for v in vs:
        print([*v.keys()])

In [ ]:
# default with a level of 0, and an indent of 4 characters
def write(p, depth=0, indent=4):
    if p==None:
        return
    # here we multiply the level by the number of indents
    # and then you multiply that number with a space character
    # which will magically show as that number of spaces.
    print("{}{}".format(" "*(indent*depth), p))
    if p.children!=None:
        # then you do not need your print(…, end='') hack
        # simply increase the depth
        write(p.children, depth=depth+1, indent=indent)

In [ ]:
def walk(node):
    """ iterate tree in pre-order depth-first search order """
   # yield node
    for child in node.children:
        print(child)
        for n in walk(child):
            yield n

In [ ]:
def text_node(node_key, tree):
    text = node_key
    children = tree[node_key].get("children")
    if not children:
        text += "\n\t"
    for child in children:
        text += "\t -->"
        text += text_node([*child.keys()][0], child)
        
    return text

In [ ]:
trees["reperfusion"]

In [ ]:
trees.keys()

In [ ]:
t = """"""
for key in trees.keys():
    t += text_node(key, trees)

In [ ]:
with open('topic_trees.txt', 'w') as f:
    print(t, file=f)

In [ ]:
walk(trees["reperfusion"])

In [ ]:

for topic, sources in td.items():
    #every combination gets overlaps
    

## Summarization

**Tried: Google Pegasus**. Result: Does a terrible job of keeping the important information and doesn't retain the question but guesses at a conclusion


In [ ]:
torch.cuda.is_available()

### Pegasus Setup


In [ ]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
import torch

model_name = 'google/pegasus-xsum'
device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(device)


In [ ]:

def summarize(sentences):
    batch = tokenizer(sentences, truncation=True, padding='longest', return_tensors="pt").to(device)
    translated = model.generate(**batch)
    tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
    return tgt_text

### T5 Setup

In [ ]:
from transformers import pipeline

summarizer = pipeline("summarization")

ARTICLE = """ Background: Trust is a critical component of competency committees given their high-stakes decisions. Research from outside of medicine on group trust has not focused on trust in group decisions, and "group trust" has not been clearly defined. The purpose was twofold: to examine the definition of trust in the context of group decisions and to explore what factors may influence trust from the perspective of those who rely on competency committees through a proposed group trust model. Methods: The authors conducted a literature search of four online databases, seeking articles published on trust in group settings. Reviewers extracted, coded, and analyzed key data including definitions of trust and factors pertaining to group trust. Results: The authors selected 42 articles for full text review. Although reviewers found multiple general definitions of trust, they were unable to find a clear definition of group trust and propose the following: a group-directed willingness to accept vulnerability to actions of the members based on the expectation that members will perform a particular action important to the group, encompassing social exchange, collective perceptions, and interpersonal trust. Additionally, the authors propose a model encompassing individual level factors (trustor and trustee), interpersonal interactions, group level factors (structure and processes), and environmental factors."""
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model = AutoModelForSeq2SeqLM.from_pretrained("snrspeaks/t5-one-line-summary") #snrspeaks/t5-one-line-summary
tokenizer = AutoTokenizer.from_pretrained("snrspeaks/t5-one-line-summary")

# T5 uses a max_length of 512 so we cut the article to 512 tokens.
inputs = tokenizer.encode("summarize: " + ARTICLE, return_tensors="pt", max_length=512, truncation=True)
outputs = model.generate(inputs, max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
tokenizer.decode(outputs[0])
def summarize(text):
    inputs = tokenizer.encode("summarize: " + text, return_tensors="pt", max_length=512, truncation=True)
    outputs = model.generate(inputs, max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
    return tokenizer.decode(outputs[0])

In [ ]:
summarize("I have seen a ghost in my shed")

In [ ]:
"I have seen a ghost in my shed"[:5]

### Print Results

In [ ]:
for ID, cluster in get_clusters(cluster_model).items():
    sentences = ".".join(cluster)
    print(sentences)
    print( "\n\n", "sum:::", summarize(sentences[:256]), "\n\n\n")
    